In [ ]:
import os
import pandas as pd;pd.__version__

In [ ]:
data_folder = r'DataOutput'
source_file = r'MarketAnalyzerSource\ExtremeReversalSignal.csv'

In [ ]:
source_df = pd.read_csv(source_file, parse_dates=['DateTime'])
tickers = source_df.Ticker.unique()
data_dict = {}

In [ ]:
bg_bar_number_name = 'Id2BarNumber'
result = []
for ticker in tickers:
    print(ticker)
    if ticker not in data_dict:
        df = pd.read_csv(os.path.join(data_folder, ticker +'.csv'), parse_dates=['TickDateTime']).set_index('TickDateTime')
        df = df.loc[~df.index.duplicated(keep='first')]
        data_dict[ticker] = df
        
    trade_df = source_df[source_df['Ticker'] == ticker].set_index('DateTime')
    for row_index, row in trade_df.iterrows():
        df =data_dict[ticker].copy(deep=True)
        matching_row_index = df[df.index >= row_index].index[0]
        bg_bar_number = df.loc[matching_row_index, bg_bar_number_name]
        trade_df.loc[row_index, 'Close'] = df.loc[matching_row_index, 'ClosePrice']
        trade_df.loc[row_index, 'ATR'] = df.loc[matching_row_index, 'Id2Atr']
        trade_df.loc[row_index, 'VAH'] = df.loc[matching_row_index, 'Id2VAH']
        trade_df.loc[row_index, 'VAL'] = df.loc[matching_row_index, 'Id2VAL']
        trade_df.loc[row_index, 'VPOC'] = df.loc[matching_row_index, 'Id2VPOC']
        trade_df.loc[row_index, 'CurrentBarPrice'] = df[df[bg_bar_number_name] == bg_bar_number].iloc[-1].loc['ClosePrice']
        trade_df.loc[row_index, 'NextBar1Price'] = df[df[bg_bar_number_name] == bg_bar_number + 1].iloc[-1].loc['ClosePrice']
        trade_df.loc[row_index, 'NextBar2Price'] = df[df[bg_bar_number_name] == bg_bar_number + 2].iloc[-1].loc['ClosePrice']
        trade_df.loc[row_index, 'NextBar3Price'] = df[df[bg_bar_number_name] == bg_bar_number + 3].iloc[-1].loc['ClosePrice']
        _df =  df[df[bg_bar_number_name].isin(list(range(bg_bar_number, bg_bar_number + 3)))]
        _df = _df[_df.index > row_index]
        min_value = _df['LowPrice'].min()
        max_value = _df['HighPrice'].max()
        if row['Direction'] == 1:
            trade_df.loc[row_index, 'MAE'] = min_value
            trade_df.loc[row_index, 'MFE'] = max_value
        else:
            trade_df.loc[row_index, 'MAE'] = max_value
            trade_df.loc[row_index, 'MFE'] = min_value
    
    result.append(trade_df)
        
        
    

In [ ]:
final_df = pd.concat(result, axis=0)

In [ ]:
final_df.to_csv('ExtremeReversalEntry.csv')